In [1]:
import openpyxl
import re
import pandas as pd

In [2]:
def excel_to_list2d(file_path: str, sheet_name: str):
    wb = openpyxl.open(filename=file_path, data_only=True)
    ws = wb[sheet_name]
    list2d = [[cell.value for cell in row] for row in ws[1:ws.max_row]]
    
    return list2d

In [3]:
def list2d_to_new_excel(list2d: list, new_path: str):
    wb = openpyxl.Workbook()
    ws = wb.active

    for row_idx, row in enumerate(list2d):
        for col_idx, col in enumerate(row):
            val = list2d[row_idx][col_idx]
            ws.cell(
                row=row_idx+1,
                column=col_idx+1,
                value=val
            )

    wb.save(new_path)

In [4]:
table = excel_to_list2d("step_5.xlsx", "Sheet")

In [5]:
donvi_count = {
    "01. Thành phố Cà Mau": 0,
    "02. Huyện Ngọc Hiển": 0,
    "03. Huyện Năm Căn": 0,
    "04. Huyện Đầm Dơi": 0,
    "05. Huyện Thới Bình": 0,
    "06. Huyện U Minh": 0,
    "07. Huyện Trần Văn Thời": 0,
    "08. Huyện Phú Tân": 0,
    "09. Huyện Cái Nước": 0,
    "01. Sở Kế hoạch và Đầu tư": 0,
    "02. Sở Xây dựng": 0,
    "03. Sở Công thương": 0,
    "04. Sở Tài nguyên và Môi trường": 0,
    "05. Sở Nông nghiệp và Phát triển nông thôn": 0,
    "06. Sở Khoa học và Công nghệ": 0,
    "07. Sở Giao thông Vận tải": 0,
    "08. Sở Lao động, Thương binh và Xã hội": 0,
    "09. Sở Văn hoá, Thể thao và Du lịch": 0,
    "10. Sở Thông tin và Truyền thông": 0,
    "11. Sở Tư pháp": 0,
    "12. Ban Quản lý Khu kinh tế": 0,
    "13. Cục Thuế": 0,
    "14. Thanh tra tỉnh": 0,
    "15. Công an tỉnh": 0,
    "16. Tòa án Nhân dân tỉnh": 0,
    "17. Bảo hiểm xã hội tỉnh": 0
}

In [6]:
for row in table:
    for i in range(34, 42+1):
        if row[i] != None:
            donvi_count[row[i]] += 1

In [7]:
donvi_series = pd.Series(donvi_count)
donvi_df = pd.DataFrame(donvi_series, columns=["count"])
donvi_df.reset_index(names=["donvi"], inplace=True)
donvi_df.sort_values(by=["count", "donvi"], ascending=True, inplace=True)
rank = 0
for idx in donvi_df.index:
    rank += 1
    donvi_df.loc[idx, "rank"] = rank
donvi_df["rank"] = donvi_df["rank"].astype(int)
donvi_df

,donvi,count,rank
22,14. Thanh tra tỉnh,0,1
24,16. Tòa án Nhân dân tỉnh,0,2
18,10. Sở Thông tin và Truyền thông,1,3
19,11. Sở Tư pháp,8,4
16,"08. Sở Lao động, Thương binh và Xã hội",40,5
20,12. Ban Quản lý Khu kinh tế,48,6
14,06. Sở Khoa học và Công nghệ,68,7
12,04. Sở Tài nguyên và Môi trường,74,8
13,05. Sở Nông nghiệp và Phát triển nông thôn,75,9
15,07. Sở Giao thông Vận tải,75,10


In [8]:
copied_tbl = table
def allocate_row(cat: str, min_row: int):
    count_row = 0
    cat_rows = []
    for i in range(34, 42+1):
        for row in table:
            if row[i] == cat and row in copied_tbl:
                cat_rows.append(row[:34]+[cat])
                copied_tbl.remove(row)
                count_row += 1
                if count_row == min_row:
                    return(cat_rows)
                else:
                    continue
    return cat_rows

In [9]:
new_tbl = []
for donvi in donvi_df["donvi"]:
    donvi_rows = allocate_row(cat=donvi, min_row=300)
    new_tbl.extend(donvi_rows)

In [10]:
for row in copied_tbl:
    new_tbl.append(row[:35])

In [11]:
list2d_to_new_excel(new_tbl, "step_6.xlsx")